In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 500     # number of variables
num_ineq = 500    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 17415.67
Epoch 1, Validation Loss: 6.64
Epoch 2, Validation Loss: 31.54
Epoch 3, Validation Loss: -31.24
Epoch 4, Validation Loss: -38.35
Epoch 5, Validation Loss: -45.83
Epoch 6, Validation Loss: -44.62
Epoch 7, Validation Loss: -48.36
Epoch 8, Validation Loss: -52.01
Epoch 9, Validation Loss: -52.34
Epoch 10, Validation Loss: -48.43
Epoch 11, Validation Loss: -51.57
Epoch 12, Validation Loss: -44.57
Epoch 13, Validation Loss: -53.28
Epoch 14, Validation Loss: -54.01
Epoch 15, Validation Loss: -53.69
Epoch 16, Validation Loss: -53.33
Epoch 17, Validation Loss: -52.28
Epoch 18, Validation Loss: -44.48
Epoch 19, Validation Loss: -38.92
Epoch 20, Validation Loss: -49.73
Epoch 21, Validation Loss: -42.83
Epoch 22, Validation Loss: -52.29
Epoch 23, Validation Loss: -47.81
Epoch 24, Validation Loss: -24.82
Epoch 25, Validation Loss: -55.92
Epoch 26, Validation Loss: -46.88
Epoch 27, Validation Loss: -54.71
Epoch 28, Validation Loss: -54.62
Epoch 29, Validation Loss

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_500-500_g.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:01<00:00,  1.63it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -60.039633          0.001496      0.006217
std      1.951367          0.008405      0.000927
min    -61.185626          0.000000      0.002998
25%    -60.562898          0.000000      0.006004
50%    -60.405182          0.000000      0.006352
75%    -60.043887          0.000000      0.006692
max    -42.056215          0.069241      0.008123
Number of infeasible solution: 4


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 6249.54
Epoch 1, Validation Loss: -4.09
Epoch 2, Validation Loss: -24.74
Epoch 3, Validation Loss: -28.29
Epoch 4, Validation Loss: -43.23
Epoch 5, Validation Loss: -45.82
Epoch 6, Validation Loss: -46.51
Epoch 7, Validation Loss: -46.12
Epoch 8, Validation Loss: -51.06
Epoch 9, Validation Loss: -47.70
Epoch 10, Validation Loss: -55.07
Epoch 11, Validation Loss: -56.86
Epoch 12, Validation Loss: -52.52
Epoch 13, Validation Loss: -56.45
Epoch 14, Validation Loss: -45.94
Epoch 15, Validation Loss: -55.00
Epoch 16, Validation Loss: -53.27
Epoch 17, Validation Loss: -53.52
Epoch 18, Validation Loss: -53.56
Epoch 19, Validation Loss: -56.38
Epoch 20, Validation Loss: -45.13
Epoch 21, Validation Loss: -57.82
Epoch 22, Validation Loss: -58.64
Epoch 23, Validation Loss: -57.66
Epoch 24, Validation Loss: -55.81
Epoch 25, Validation Loss: -56.32
Epoch 26, Validation Loss: -52.54
Epoch 27, Validation Loss: -51.18
Epoch 28, Validation Loss: -53.73
Epoch 29, Validation Los

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_500-500_g.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:00<00:00,  1.65it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -57.507296          0.000253      0.006254
std      0.728495          0.002527      0.001035
min    -58.260573          0.000000      0.003101
25%    -57.846866          0.000000      0.005999
50%    -57.590366          0.000000      0.006345
75%    -57.412914          0.000000      0.006843
max    -52.791393          0.025274      0.008092
Number of infeasible solution: 1
